### Outline:

0. Load libraries and custom functions.
1. Load data.
2. Preliminary data analysis: explore features and a target, delete unneeded features, create new features.
3. Train-test split.
4. Missing values. In some cases it may be useful to explore skew and perform log-transform before imputing missing values.
5. Feature engineering. Transform skewed variables, do OHC and scaling.
6. Fit models.
7. Evaluate models.
8. Feature importance, error analysis. Based on the results, go to 2. and iterate.
9. Make predictions.

In [56]:
import numpy as np
import pandas as pd
import os, time, warnings, random
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_curve
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier

pd.set_option('display.max_columns', 20)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings('ignore')

def draw_histograms(df, variables, n_rows, n_cols):
    # stolen from https://stackoverflow.com/questions/29530355/plotting-multiple-histograms-in-grid
    fig=plt.figure()
    for i, var_name in enumerate(variables):
        ax=fig.add_subplot(n_rows,n_cols,i+1)
        df[var_name].hist(bins=10,ax=ax)
        ax.set_title(var_name+" Distribution")
    fig.tight_layout()  
    plt.show()


def fillna_mp_i1(df_train, df_test, df_pred, num_features, cat_features, num_fill='median', cat_fill='mode'):
    """This function speeds up filling missing values for 3 main datasets using different imputation methods.
    Later may replace it with some subclass.
    Example: fillna_mp_i1(X_train, X_test, X_pred, num_cols, cat_cols)"""
    # set df_pred to None if it does not exist
    if not ((cat_fill=='mode') and (num_fill=='median')):
        print ('Imputation method not Implemented yet!')
        return None
    
    df_train[num_features] = df_train[num_features].fillna(value=df_train[num_features].median())
    df_test[num_features] = df_test[num_features].fillna(value=df_train[num_features].median())
    df_train[cat_features] = df_train[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
    df_test[cat_features] = df_test[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
    if (df_pred is not None):
        df_pred[num_features] = df_pred[num_features].fillna(value=df_train[num_features].median())
        df_pred[cat_features] = df_pred[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
    df_train[num_features+cat_features].count
    
    all_good = (
    (np.prod(df_train[num_features+cat_features].shape)==df_train[num_features+cat_features].count().sum()) and 
    (np.prod(df_test[num_features+cat_features].shape) == df_test[num_features+cat_features].count().sum()) and 
    (np.prod(df_pred[num_features+cat_features].shape) == df_pred[num_features+cat_features].count().sum()))
    if (all_good):
        print('Missing values imputed successfully')
    else:
        print('There are still some missing values...')
    
def add_misDummy_mp_i1(df_train, df_test, df_pred, features):
    """This function creates new dummy columns for missing features.
    Example: add_misDummy_mp_i1(X_train, X_test, X_pred, ['Age'])"""
    # set df_pred to None if it does not exist
    for feature_name in features:
        misColName = 'mis'+feature_name
        df_train.loc[df_train[feature_name].isnull(), misColName]=1
        df_train.loc[df_train[feature_name].notnull(), misColName]=0
        df_test.loc[df_test[feature_name].isnull(), misColName]=1
        df_test.loc[df_test[feature_name].notnull(), misColName]=0
        if (df_pred is not None):
            df_pred.loc[df_pred[feature_name].isnull(), misColName]=1
            df_pred.loc[df_pred[feature_name].notnull(), misColName]=0
   

def discretize_mp_i1(df_train, df_test, df_pred, feature, ntiles, delete_feature=False):
    """This function divides a continuous feature into quantile groups.
    Example: discretize_mp_i1(X_train, X_test, X_pred, 'Age', 15)"""
    # set df_pred to None if it does not exist
    _,bin = pd.qcut(df_train[feature], ntiles, retbins = True, labels = False, duplicates = 'drop')
    df_train[feature+'Ntile'] = pd.cut(df_train[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    df_test[feature+'Ntile'] = pd.cut(df_test[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    if (df_pred is not None):
        df_pred[feature+'Ntile'] = pd.cut(df_pred[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    if (delete_feature==True):
        df_train.drop(columns=[feature], inplace=True)
        df_test.drop(columns=[feature], inplace=True)
        df_pred.drop(columns=[feature], inplace=True)
    print('Discretized ',feature, ' into ', len(bin)-1, ' bins')


def log_transformer_mp_i1(df_train, df_test, df_pred, feature_subset=False, min_skew=3):
    """This function divides a continuous feature into quantile groups.
    Example: log_transformer_mp_i1(X_train, X_test, X_pred, feature_subset=num_cols)"""
    # set df_pred to None if it does not exist
    if (feature_subset==False):
        features_totransform = df_train.columns
    else:
        features_totransform = feature_subset.copy()
    skewed_vars = list(df_train.skew()[abs(df_train.skew())>min_skew].index)
    for col in list(set(skewed_vars)&set(features_totransform)):
        df_train[col] = np.log1p(df_train[col])
        df_test[col] = np.log1p(df_test[col])
        if (df_pred is not None):
            df_pred[col] = np.log1p(df_pred[col])
    print('Skewed columns log-transformed: ', list(set(skewed_vars)&set(features_totransform)))
    
    
def add_dummyfeatures(df_train, df_test, df_pred, feature_dict):
    """This function adds dummy feature when some feature is equal to value, specified in a dictionary.
    Example: add_dummyfeatures(X_train, X_test, X_pred, {'RoomService':0, 'Spa':0, 'VRDeck':0, 'ShoppingMall':0})"""
    input_dimensions = np.array([df_train.shape[1], df_test.shape[1], df_pred.shape[1]])
    for i in range(len(list(feature_dict.items()))):
        feature,value = list(feature_dict.keys())[i], list(feature_dict.values())[i]
        df_train.loc[df_train[feature]==value,(str(feature)+str(value))]=1
        df_train.loc[df_train[feature]!=value,(str(feature)+str(value))]=0
        df_test.loc[df_test[feature]==value,(str(feature)+str(value))]=1
        df_test.loc[df_test[feature]!=value,(str(feature)+str(value))]=0
        df_pred.loc[df_pred[feature]==value,(str(feature)+str(value))]=1
        df_pred.loc[df_pred[feature]!=value,(str(feature)+str(value))]=0
    output_dimensions = np.array([df_train.shape[1], df_test.shape[1], df_pred.shape[1]])
    print(output_dimensions-input_dimensions, ' variables created') 
    

In [44]:
# 1. Import data #

time0 = time.time()

path = '../input/santander-customer-transaction-prediction/train.csv'
df = pd.read_csv(path) 
df0 = df.copy()
#df = df.sample(50000)

#df.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId'],inplace=True)
pred=pd.read_csv('../input/santander-customer-transaction-prediction/test.csv')
pred0 = pred.copy()
#pred.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId'],inplace=True)

print(df.shape, pred.shape)
print(df.target.mean())
# unbalanced responsed variable
df.head()

(200000, 202) (200000, 201)
0.10049


,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [45]:
# 2. EDA #

# with all features unnamed normally-distributed features, there is not much EDA and feature engineering to do.
df.drop(columns = ['ID_code'], inplace= True)

# 3. Train-test split #

train_y = df[['target']]
train_x = df.drop(columns = ['target'])
X_pred = pred.copy()

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size = 0.02, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, X_pred.shape)

X_train.count().sum() == np.prod(X_train.shape)
# no missing values, good.

(196000, 200) (4000, 200) (196000, 1) (200000, 201)


True

In [46]:
# 5. feature engineering #

ss = StandardScaler()

for col in X_train.columns:
    X_train[[col]] = ss.fit_transform(X_train[[col]])
    X_test[[col]] = ss.transform(X_test[[col]])
    X_pred[[col]] = ss.transform(X_pred[[col]])

#X_test.iloc[:,:30].describe()
random.seed(1)
fewfeatures = random.sample(list(X_train.columns),5)

In [6]:
fewfeatures

['var_34', 'var_145', 'var_195', 'var_16', 'var_65']

In [47]:
# 6. Fit models #

time1 = time.time()
lr = LogisticRegression()
param_grid = {'C':[0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30]}
lrm = GridSearchCV(lr, param_grid, cv=2, scoring='f1')
lrm.fit(X_train, y_train)
print('Logistic ', lrm.best_params_, lrm.best_score_, f1_score(y_train, lrm.predict(X_train)), time.time()-time1)

Logistic  {'C': 1} 0.3882953318402199 0.3897663585413786 14.249778509140015


In [42]:
time1 = time.time()
knn = KNeighborsClassifier(n_jobs=-1)
param_grid = dict(n_neighbors=range(10, 41, 10))
knnm = GridSearchCV(knn, param_grid, cv=2)
knnm.fit(X_train[fewfeatures], y_train)
print('KNN ', knnm.best_params_, f1_score(y_train, knnm.predict(X_train[fewfeatures])), time.time()-time1)

KeyboardInterrupt: 

In [33]:
time1 = time.time()
rf = RandomForestClassifier(n_jobs=-1)
param_grid = {'n_estimators':[100], 'max_depth':[4,5,6], 'max_features':[10]}
rfm = GridSearchCV(rf, param_grid, cv=2, scoring = 'f1')
rfm.fit(X_train, y_train)
print('RF ', rfm.best_params_, rfm.best_score_, f1_score(y_train, rfm.predict(X_train)), time.time()-time1)

RF  {'max_depth': 4, 'max_features': 10, 'n_estimators': 100} 0.0 0.0 59.155372619628906


In [48]:
time1 = time.time()
nb = GaussianNB()
nb.fit(X_train, y_train)
print('RF ', f1_score(y_train, nb.predict(X_train)), time.time()-time1)

RF  0.4854427414497022 1.2347526550292969


In [54]:
time1 = time.time()
xgb = XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=3, n_jobs=-1)
param_grid = {'n_estimators':[200], 'max_depth':[2,3], 'eta':[0.35],
'subsample':[0.3],'colsample_bytree':[0.6]}
xgbm = GridSearchCV(xgb, param_grid, cv=2, scoring='f1')
xgbm.fit(X_train, y_train)
print('XGB ', xgbm.best_params_, xgbm.best_score_, f1_score(y_train, xgbm.predict(X_train)), time.time()-time1)


XGB  {'colsample_bytree': 0.6, 'eta': 0.35, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.3} 0.45578656783654314 0.5263783426873555 18.169593572616577


In [50]:
# 7. accuracy #

print('Out of Sample:')
print('Logistic ', f1_score(y_test, lrm.predict(X_test)))
#print('SVM ', accuracy_score(y_test, svmm.predict(X_test)))
#print('KNN ', accuracy_score(y_test, knnm.predict(X_test[fewfeatures])))
print('Bayes ', f1_score(y_test, nb.predict(X_test)))
#print('RF ', f1_score(y_test, rfm.predict(X_test)))
print('XGB ', f1_score(y_test, xgbm.predict(X_test)))
print('Total time ', time.time()-time0)

Out of Sample:
Logistic  0.36121673003802285
Bayes  0.47755834829443444
XGB  0.4618055555555556
Total time  90.7634391784668


In [52]:
# VotingClassifier:

estimator = []
estimator.append(('LR', LogisticRegression(C=10)))
estimator.append(('NB', GaussianNB()))
#estimator.append(('RF', RandomForestClassifier(max_depth=4, max_features=10, n_estimators=100)))
estimator.append(('XGB', XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=3, n_jobs=-1,
                                       eta=0.35, max_depth=3, n_estimators=200, 
                                       subsample=0.3, colsample_bytree=0.6)))
vot_soft = VotingClassifier(estimators = estimator, voting ='soft')
vot_soft.fit(X_train, y_train)
vot_hard = VotingClassifier(estimators = estimator, voting ='hard')
vot_hard.fit(X_train, y_train)
print('VotingClassifiers3 in sample', f1_score(y_train, vot_soft.predict(X_train)), f1_score(y_train, vot_hard.predict(X_train)))
print('VotingClassifiers3 out of sample', f1_score(y_test, vot_soft.predict(X_test)), f1_score(y_test, vot_hard.predict(X_test)))

VotingClassifiers3 in sample 0.47543940304841387 0.47184330185379497
VotingClassifiers3 out of sample 0.46927374301675984 0.44981412639405205


In [55]:
# 8. feature importance #

results = permutation_importance(xgbm, X_test, y_test, scoring='f1', n_jobs=-1)
fi_lr = pd.DataFrame({'col':X_test.columns, 'FI':results.importances_mean})
fi_lr.sort_values('FI', ascending = False)[:20]

,col,FI
22,var_22,0.023636
169,var_169,0.021585
6,var_6,0.021046
53,var_53,0.020804
13,var_13,0.018807
149,var_149,0.016455
2,var_2,0.015433
191,var_191,0.015149
139,var_139,0.015090
34,var_34,0.014051


In [41]:
fi_lr.sort_values('FI', ascending = False)[:20]

,col,FI
53,var_53,0.039975
174,var_174,0.038795
49,var_49,0.035439
76,var_76,0.034309
133,var_133,0.034294
125,var_125,0.032684
89,var_89,0.029413
81,var_81,0.029276
92,var_92,0.029152
139,var_139,0.028353


In [42]:
fi_lr0 = fi_lr.copy()

In [43]:
fi_lr0

,col,FI
0,var_0,0.009333
1,var_1,0.019577
2,var_2,0.021051
3,var_3,-0.001511
4,var_4,0.000000
...,...,...
195,var_195,0.002108
196,var_196,-0.003104
197,var_197,-0.014163
198,var_198,-0.005287
